In [164]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager

#import pandas to read html
import pandas as pd

In [165]:
executable_path = {'executable_path': ChromeDriverManager(version="98.0.4758.102").install()}
browser = Browser('chrome', **executable_path, headless=False)




====== WebDriver manager ======
Trying to download new driver from https://chromedriver.storage.googleapis.com/98.0.4758.102/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\Eli Solomon\.wdm\drivers\chromedriver\win32\98.0.4758.102]


In [166]:
# Visit the mars nasa news site
url = 'https://redplanetscience.com'
browser.visit(url)
# Optional delay for loading the page
browser.is_element_present_by_css('div.list_text', wait_time=1)

True

In [167]:
#set up HTML parser
html = browser.html
news_soup = soup(html, 'html.parser')
slide_elem = news_soup.select_one('div.list_text')

In [168]:
slide_elem

<div class="list_text">
<div class="list_date">March 13, 2022</div>
<div class="content_title">Meet the People Behind NASA's Perseverance Rover</div>
<div class="article_teaser_body">These are the scientists and engineers who built NASA's next Mars rover and who will guide it to a safe landing in Jezero Crater. </div>
</div>

In [169]:
# Use the parent element to find the first `a` tag and save it as `news_title`
news_title = slide_elem.find('div', class_='content_title').get_text()
news_title


"Meet the People Behind NASA's Perseverance Rover"

In [170]:
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_='article_teaser_body').get_text()
news_p

"These are the scientists and engineers who built NASA's next Mars rover and who will guide it to a safe landing in Jezero Crater. "

### Featured Images

In [171]:
# Visit URL
url = 'https://spaceimages-mars.com'
browser.visit(url)

In [172]:
# Find and click the full image button
full_image_elem = browser.find_by_tag('button')[1] #indexing changed to click second button
full_image_elem.click()

In [173]:
# Parse the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')

In [174]:
# Find the relative image url
img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
img_url_rel

'image/featured/mars3.jpg'

In [175]:
# Use the base URL to create an absolute URL
img_url = f'https://spaceimages-mars.com/{img_url_rel}'
img_url

'https://spaceimages-mars.com/image/featured/mars3.jpg'

### Scraping an HTML table

In [176]:
#read_html function returns a list of tables found in the HTML, index 0 pulls first table it incounters
df = pd.read_html('https://galaxyfacts-mars.com')[0]
#set up the dataframe
df.columns=['description', 'Mars', 'Earth']
#change the description column into the DF index
df.set_index('description', inplace=True)
df

,Mars,Earth
description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [177]:
#convert the DF back to html
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>

## D1: Scraope High-Resolution Mars' Hemisphere Images and Titles

### Hemispheres

In [178]:
# 1. Use browser to visit the URL 
url = 'https://marshemispheres.com/'

browser.visit(url)

In [179]:
# 2. Create a list to hold the images and titles.
hemisphere_image_urls = []

# 3. Write code to retrieve the image urls and titles for each hemisphere.
html =  browser.html
soup = soup(html, "html.parser")
#hemisphere_image_urls = soup.find_all("a", class_='itemLink product-item')
div_items = soup.find_all("div", class_ = 'item')
len(div_items)

4

In [180]:
div_items[0].a.get("href")

'cerberus.html'

In [181]:
#loop through array of each div_items hemisphere 
for div_item in div_items:
    #kept haveing to reimport soup in testing
    from bs4 import BeautifulSoup as soup
    
    #empty dic to store scrape
    hemisphere = {'img_url':'','title':''}
    #print relative link
    #print(div_item.a.get("href"))
    #append the relative link onto url
    img_url = url + div_item.a.get("href")
    #print(img_url)
    #tell splinter to visit each url
    browser.visit(img_url)
    html_sub = browser.html
    soup = soup(html_sub, "html.parser")
    relative_jpeg_url = soup.find("li").a.get("href")
    jpeg_url = url + relative_jpeg_url
    #print(jpeg_url)
    
    hemisphere['img_url']=(jpeg_url)
    
    title = soup.find("h2", class_="title").text #.split(" | ")
    #print(title)
    
    hemisphere['title']=(title)
    #print(hemisphere)
    
    hemisphere_image_urls.append(hemisphere)
    #print("-----------")

#test prints
#import pprint
#pprint.pprint(hemisphere_image_urls)

In [182]:
#print(hemisphere_image_urls[0].find('img')["src"])

In [183]:
# 4. Print the list that holds the dictionary of each image url and title.
hemisphere_image_urls

[{'img_url': 'https://marshemispheres.com/images/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

In [184]:
#end the splinter browser
browser.quit()